# Get SLR projections for each tide gauge

In [14]:
import pandas as pd
import numpy as np

In [7]:
# data paths
raw_data = "../data/raw/"
clean_data = "../data/clean/"

all_slr = pd.read_csv(raw_data + "slr/SLR_TF U.S. Sea Level Projections.csv", skiprows=17)

Clean up NOAA SLR estimates to use for SLAMM

In [15]:
# filter to just the sites we want
wanted_slr = all_slr[all_slr['NOAA Name'].isin(['GMSL', 'Woods Hole', 'Providence'])]

# tidy up
wanted_slr = wanted_slr.drop(['PSMSL Site',	'PSMSL ID','RSL GridNum', 'Lat', 'Long',	
                              'Regional Classification', 'US Coastline Intersect (1 = yes)', 
                              'Offset 1992 to 2000 (cm)', 'RSL contribution from VLM (trend: cm/year)'],  axis = 1)
wanted_slr = pd.melt(wanted_slr, 
                     id_vars=['NOAA ID', 'NOAA Name', 'Scenario',
                               'Offset 2000 to 2005 (cm)'], 
                    value_vars=wanted_slr.columns[wanted_slr.columns.str.startswith('RSL')],
                    var_name='year',
                    value_name='RSL (cm)')

# create year column and remove the "RSL (cm)"" part 
wanted_slr['year'] = wanted_slr['year'].str.extract(r'(\d+)', expand=False)

# To initiate the projections at year 2000, the value of the 2000-2005 offset should be added to each projection value 
wanted_slr['year'] = ['2000' if a == '2005' else a for a in wanted_slr['year']] # first set 2005 as 2000
wanted_slr['RSL (cm)'] = [0 if a == 0 else a + 2 for a in wanted_slr['RSL (cm)']]

# add a mm column
wanted_slr['RSL (m)'] = wanted_slr['RSL (cm)'] / 100
wanted_slr

,NOAA ID,NOAA Name,Scenario,Offset 2000 to 2005 (cm),year,RSL (cm),RSL (m)
0,0.0,GMSL,0.3 - MED,2,2000,0,0.00
1,0.0,GMSL,0.3 - LOW,2,2000,0,0.00
2,0.0,GMSL,0.3 - HIGH,2,2000,0,0.00
3,0.0,GMSL,0.5 - MED,2,2000,0,0.00
4,0.0,GMSL,0.5 - LOW,2,2000,0,0.00
...,...,...,...,...,...,...,...
670,8454000.0,Providence,1.5 - LOW,2,2150,181,1.81
671,8454000.0,Providence,1.5 - HIGH,2,2150,365,3.65
672,8454000.0,Providence,2.0 - MED,2,2150,343,3.43
673,8454000.0,Providence,2.0 - LOW,2,2150,242,2.42


EPA wants medium and high for slr scenarios 

In [16]:
wanted_slr.Scenario.unique()

array(['0.3 - MED', '0.3 - LOW', '0.3 - HIGH', '0.5 - MED', '0.5 - LOW',
       '0.5 - HIGH', '1.0 - MED', '1.0 - LOW', '1.0 - HIGH', '1.5 - MED',
       '1.5 - LOW', '1.5 - HIGH', '2.0 - MED', '2.0 - LOW', '2.0 - HIGH'],
      dtype=object)

In [16]:
def get_slr(wanted_slr: pd.DataFrame, station: str, scenario: str, filename:str):
    station_slr = wanted_slr[wanted_slr['NOAA Name'] == station]
    station_slr = station_slr[station_slr['Scenario'] == scenario]
    station_slr = station_slr[["year", "RSL (m)"]]
    station_slr.to_csv(clean_data + "slr/" + filename + "_2000_base.csv")

    return station_slr

Global SLR

In [17]:
# 1 medium scenario
global_01_med = get_slr(wanted_slr, 'GMSL', '1.0 - MED', 'global_01_med')
global_02_med = get_slr(wanted_slr, 'GMSL', '2.0 - MED', 'global_02_med')

Woodshole

In [5]:
# 0.5 medium scenario
wh_05_med = get_slr(wanted_slr, 'Woods Hole', '0.5 - MED', 'wh_05_med')
prov_05_med = get_slr(wanted_slr, 'Providence', '0.5 - MED', 'prov_05_med')

In [18]:
# 1 medium scenario
wh_01_med = get_slr(wanted_slr, 'Woods Hole', '1.0 - MED', 'wh_01_med')
prov_01_med = get_slr(wanted_slr, 'Providence', '1.0 - MED', 'prov_01_med')

In [19]:
# 2 medium scenario
wh_02_med = get_slr(wanted_slr, 'Woods Hole', '2.0 - MED', 'wh_02_med')
prov_02_med = get_slr(wanted_slr, 'Providence', '2.0 - MED', 'prov_02_med')